### Setup

In [1]:
import re
import os
import pandas as pd
path_master = r'C:/Users/endwy/Documents/Columbia MSBA/Spring 2019/E4524 - Analytics in Practice/Data/Csv/'
file_names = list()
for path, subdirs, files in os.walk(path_master):
    for filename in files:
        file_names.append(filename)
df_final = pd.DataFrame()

### Functions to read in CSVs

In [2]:
def add_time(df_ABR1):
    """Add Month and Year column based on read-in file name"""
    mon=re.search(r'\w+', file_name)
    df_ABR1['Month']= file_name[mon.start():mon.end()]
    y=re.search(r'\d+', file_name)
    df_ABR1['Year'] = file_name[y.start():y.end()]
    
def type1(pm,tt,fn,d='.csv'):
    """10 columns, missing 7,9, concatenate Mortalities column"""
    with open(pm+tt+fn+d) as fp:
        df_ABR1 = pd.read_csv(fp, 
                              usecols=['0','1','2','3','4','5','6','8','10'],
                              keep_default_na=False)
    df_ABR1.rename(columns={'0':'Drug', '1':'Treated', '2':'Repulls', '3':'Mortalities', '4':'MortalitiesB', 
                        '5':'Treatment Success(%)', '6':'Case Fatality Rate(%)', '8':'$/Hd/Rx','10':'Treatment $'}, inplace=True)
    # Concatenate mortalities column
    for i in range(len(df_ABR1)):
        df_ABR1.iloc[i].loc['Mortalities'] = (df_ABR1.iloc[i].loc['Mortalities'])+(df_ABR1.iloc[i].loc['MortalitiesB'])
    del df_ABR1['MortalitiesB']
    add_time(df_ABR1)
    global df_final
    df_final = pd.concat([df_final,df_ABR1])

def type2(pm,tt,fn,d='.csv'):
    """9 columns, missing 6,8"""
    with open(pm+tt+fn+d) as fp:
        df_ABR1 = pd.read_csv(fp,
                              usecols=['0','1','2','3','4','5','7','9'], keep_default_na=False)
    df_ABR1.rename(columns={'0':'Drug', '1':'Treated', '2':'Repulls', '3':'Mortalities', '4':'Treatment Success(%)', 
                            '5':'Case Fatality Rate(%)', '7':'$/Hd/Rx', '9':'Treatment $',}, 
                       inplace=True)
    add_time(df_ABR1)
    global df_final
    df_final = pd.concat([df_final,df_ABR1])

def type3(pm,tt,fn,d='.csv'):
    """base case"""
    with open(pm+tt+fn+d) as fp:
        df_ABR1 = pd.read_csv(fp,
                          skiprows=[0], 
                          usecols=['Unnamed: 1', 'Treated', 'Repulls', 'Mortalities', 'Treatment Success',
                                   'Case Fatality Rate','$/Hd/Rx', 'Treatment $'])
    df_ABR1.rename(columns={'Unnamed: 1': 'Drug', 'Treatment Success':'Treatment Success(%)',
                        'Case Fatality Rate':'Case Fatality Rate(%)'}, inplace=True)
    add_time(df_ABR1)
    global df_final
    df_final = pd.concat([df_final,df_ABR1])

def type4(pm,tt,fn,d='.csv'):
    """11 columns, missing 7,9, concatenate Treatment column"""
    with open(pm+tt+fn+d) as fp:
                df_ABR1 = pd.read_csv(fp,
                         usecols=['0','1','2','3','4','5','6','8','10','11'],
                         keep_default_na=False)
    df_ABR1.rename(columns={'0':'Drug', '1':'Treated', '2':'Repulls', '3':'RepullsB', '4':'Mortalities', 
                        '5':'Treatment Success(%)', '6':'Case Fatality Rate(%)', '8':'$/Hd/Rx',
                        '10':'Treatment $','11':'Treatment $B'}, inplace=True)
    # Concatenate Treatment $ column
    for i in range(len(df_ABR1)):
        df_ABR1.iloc[i].loc['Treatment $'] = (df_ABR1.iloc[i].loc['Treatment $'])+(df_ABR1.iloc[i].loc['Treatment $B'])
    del df_ABR1['Treatment $B']
    for i in range(len(df_ABR1)):
        df_ABR1.iloc[i].loc['Repulls'] = (df_ABR1.iloc[i].loc['Repulls'])+(df_ABR1.iloc[i].loc['RepullsB'])
    del df_ABR1['RepullsB']
    add_time(df_ABR1)
    global df_final
    df_final = pd.concat([df_final,df_ABR1])

def type5(pm,tt,fn,d='.csv'):
    """10 column, missing 6,8, concatenate Treatment column"""
    with open(pm+tt+fn+d) as fp:
                df_ABR1 = pd.read_csv(fp,
                         usecols=['0','1','2','3','4','5','7','9','10'],
                         keep_default_na=False)
    df_ABR1.rename(columns={'0':'Drug', '1':'Treated', '2':'Repulls', '3':'Mortalities', '4':'Treatment Success(%)', 
                        '5':'Case Fatality Rate(%)', '7':'$/Hd/Rx', 
                        '9':'Treatment $','10':'Treatment $B'}, inplace=True)
    # Concatenate Treatment $ column
    for i in range(len(df_ABR1)):
        df_ABR1.iloc[i].loc['Treatment $'] = (df_ABR1.iloc[i].loc['Treatment $'])+(df_ABR1.iloc[i].loc['Treatment $B'])
    del df_ABR1['Treatment $B']
    add_time(df_ABR1)
    global df_final
    df_final = pd.concat([df_final,df_ABR1])

def type6(pm,tt,fn,d='.csv'):
    """11 column, concatenate Treatment column"""
    with open(path_master+title+file_num+'.csv') as fp:
                df_ABR1 = pd.read_csv(fp,
                         usecols=['0','1','3','4','5','6','7','8','9','10','11'],
                         keep_default_na=False)
    df_ABR1.rename(columns={'0':'Drug', '1':'Treated', '3':'Repulls', '4':'Mortalities','5':'MortalitiesB','6':'Treatment Success(%)', 
                        '7':'Case Fatality Rate(%)', '8':'$/Hd/Rx', '9':'$/Hd/RxB',
                        '10':'Treatment $','11':'Treatment $B'}, inplace=True)
    # Concatenate Treatment $ column
    for i in range(len(df_ABR1)):
        df_ABR1.iloc[i].loc['Mortalities'] = (df_ABR1.iloc[i].loc['Mortalities'])+(df_ABR1.iloc[i].loc['MortalitiesB'])
        df_ABR1.iloc[i].loc['$/Hd/Rx'] = (df_ABR1.iloc[i].loc['$/Hd/Rx'])+(df_ABR1.iloc[i].loc['$/Hd/RxB'])
        df_ABR1.iloc[i].loc['Treatment $'] = (df_ABR1.iloc[i].loc['Treatment $'])+(df_ABR1.iloc[i].loc['Treatment $B'])
    del df_ABR1['MortalitiesB']
    del df_ABR1['$/Hd/RxB']
    del df_ABR1['Treatment $B']
    df_ABR1['Treated']=df_ABR1['Treated'].apply(lambda x: x.replace(' ',''))
    add_time(df_ABR1)
    global df_final
    df_final = pd.concat([df_final,df_ABR1])

def type7(pm,tt,fn,d='.csv'):
    """11 columns, missing 8,10"""
    with open(path_master+title+file_num+'.csv') as fp:
                df_ABR1 = pd.read_csv(fp,
                         usecols=['0','1','2','4','6','7','9','11'], keep_default_na=False)
    df_ABR1.rename(columns={'0':'Drug', '1':'Treated', '2':'Repulls', '4':'Mortalities', '6':'Treatment Success(%)', 
                        '7':'Case Fatality Rate(%)', '9':'$/Hd/Rx', '11':'Treatment $',
                        }, inplace=True)
    add_time(df_ABR1)
    global df_final
    df_final = pd.concat([df_final,df_ABR1])

In [ ]:
# APRIL 2017 & 2018
for file_name in file_names:
    pattern = r'April\s+\d+\s+\-\s+Antibiotic by Risk'
    title = file_name.split("Risk")[0]+str("Risk")
    if re.search(pattern=pattern, string=file_name):
        file_num=re.findall(r'\d+', file_name)[-1]
        # cleaning for 2017
        if file_name.split()[1] == '2017':
            if file_num == '3' or file_num == '5' or file_num == '8':
                type1(path_master,title,file_num)
            elif file_num == '4' or file_num == '9':
                type2(path_master,title,file_num)
            elif file_num == '1':
                type3(path_master,title,file_num)
            elif file_num == '2':
                type4(path_master,title,file_num)
            elif file_num == '6':
                type5(path_master,title,file_num)
            elif file_num == '7':
                type6(path_master,title,file_num)
            elif file_num == '10':
                type7(path_master,title,file_num)
        # cleaning for 2018
        elif file_name.split()[1] == '2018':
            if file_num == '1':
                type3(path_master,title,file_num)
            elif file_num == '2':
                type5(path_master,title,file_num)
            elif file_num == '3' or file_num=='4' or file_num=='6' or file_num=='7' or file_num=='10':
                type1(path_master,title,file_num)  
            elif file_num == '3' or file_num=='5' or file_num=='9':
                type2(path_master,title,file_num)
            elif file_num == '8':
                type7(path_master,title,file_num)

# JANUARY 2017 & 2018 (note: no 2017 file)
for file_name in file_names:
    pattern = r'January\s+\d+\s+\-\s+Antibiotic by Risk'
    title = file_name.split("Risk")[0]+str("Risk")
    if re.search(pattern=pattern, string=file_name):
        file_num=re.findall(r'\d+', file_name)[-1]
        # cleaning for 2018
        if file_name.split()[1] == '2018':
            if file_num == '1':
                type3(path_master,title,file_num)
            elif file_num == '2':
                type4(path_master,title,file_num)
            elif file_num == '3' or file_num == '5' or file_num == '8'or file_num == '10':
                type1(path_master,title,file_num)
            elif file_num == '4':
                type5(path_master,title,file_num)
            elif file_num == '6' or file_num == '7'or file_num == '9':
                type2(path_master,title,file_num)    

In [ ]:
# FEBRUARY 2017 & 2018
for file_name in file_names:
    pattern = r'February\s+\d+\s+\-\s+Antibiotic by Risk'
    title = file_name.split("Risk")[0]+str("Risk")
    if re.search(pattern=pattern, string=file_name):
        file_num=re.findall(r'\d+', file_name)[-1]
        # cleaning for 2018
        if file_name.split()[1] == '2017':
            if file_num == '1':
                type3(path_master,title,file_num)
            elif file_num == '2':
                type4(path_master,title,file_num)
            elif file_num == '3' or file_num == '5'or file_num == '8':
                type1(path_master,title,file_num)
            elif file_num == '4'or file_num == '9':
                type2(path_master,title,file_num)
            elif file_num == '6':
                type5(path_master,title,file_num)
            elif file_num == '7'or file_num == '10':
                type7(path_master,title,file_num)
        elif file_name.split()[1] == '2018':
            if file_num == '1':
                type3(path_master,title,file_num)
            elif file_num == '2':
                type4(path_master,title,file_num)
            elif file_num == '3':
                type1(path_master,title,file_num)
            elif file_num == '4':
                type5(path_master,title,file_num)

In [ ]:
b = df_final.loc[df_final['Year'] == '2018']
#len(b)


### To test which function 'type' to use

In [25]:
import pandas as pd
with open(path_master+'February 2018 - Antibiotic by Risk5.csv') as fp:
    df_ABR1 = pd.read_csv(fp)
df_ABR1.head(7)

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10
0,0,High Risk,"19,134","3,246",1.0,",805",83.04%,9.43%,$25.85,NaN,$,"4,365,673.22"
1,1,300 PRO LA,-,-,NaN,-,0.00%,0.00%,$-,NaN,$,-
2,2,A180,-,-,NaN,-,0.00%,0.00%,$-,NaN,$,-
3,3,A180 As 2nd Rx,-,-,NaN,-,0.00%,0.00%,$-,NaN,$,-
4,4,Adspec,-,-,NaN,-,0.00%,0.00%,$-,NaN,$,-
5,5,Advocin,503,43,NaN,213,91.45%,42.35%,$40.81,NaN,$,"20,526.42"
6,6,Advocin As 2nd Rx,-,-,NaN,-,0.00%,0.00%,$-,NaN,$,-


In [ ]:
df_ABR.head(5)

In [ ]:
# Save out df_ABR
import csv

# writer = csv.writer(open(path_master+"/OutputCSV.csv", 'wb'))
df_ABR.to_csv(path_master+"/df_ABR.csv")
# for row in df_ABR:
#     writer.writerow(row)

In [ ]:
# Below is not to be used yet. To clean df_ABR

def convert_1(x):
    if re.search(pattern = r'\w+',string=x):
        return x
    x=x.strip()
    if x[-1]=='%':
        return float(x.replace('%,'))
    if x=='-':
        return 0
    if x[0]=='$':
        return float(x[1:].replace(',','').replace('-','0'))

def ABR_Clean(df):
    """Cleans fully merged df_ABR columns to ensure appropriate dtypes/formats"""
    for col in df.columns:
        df[col]=df[col].apply(convert_1)
    
# Run this below to convert to .py file before pushing to GitHub
get_ipython().system('jupyter nbconvert --to script Antibiotics_by_Risk.ipynb')

In [ ]:
# Clean FULL df_ABR
df_ABR = df_final
for i in range(len(df_ABR)):
    for j in ['Treated','Repulls','Mortalities']:
        if df_ABR.iloc[i].loc[j] == '-':
            df_ABR.iloc[i].loc[j] = 0
        else:
            m = df_ABR.iloc[i].loc[j]
            m = m.replace(' ','')
            df_ABR.iloc[i].loc[j] = m.replace(',','')

# Change str -> int for numerical columns
# df_ABR['Treated'] = df_ABR['Treated'].apply(lambda x: int(x))
# df_ABR['Repulls'] = df_ABR['Repulls'].apply(lambda x: int(x))
# df_ABR['Mortalities'] = df_ABR['Mortalities'].apply(lambda x: int(x))

df_ABR['Treated'] = df_ABR['Treated'].convert_objects(convert_numeric=True)
df_ABR['Repulls'] = df_ABR['Repulls'].convert_objects(convert_numeric=True)
df_ABR['Mortalities'] = df_ABR['Mortalities'].convert_objects(convert_numeric=True)

# Remove dollar signs from columns
df_ABR['Treatment $'] = df_ABR['Treatment $'].apply(lambda x: x[1:])
df_ABR['$/Hd/Rx'] = df_ABR['$/Hd/Rx'].apply(lambda x: x[1:])

# Remove dashes from columns
for i in range(len(df_ABR)):
    for j in ['$/Hd/Rx','Treatment $']:
        if df_ABR.iloc[i].loc[j] == '-':
            df_ABR.iloc[i].loc[j] = 0.0
        else:
            m = df_ABR.iloc[i].loc[j]
            df_ABR.iloc[i].loc[j] = m.replace(',','')
            m = m.replace(' ','')
            df_ABR.iloc[i].loc[j] = m.replace(',','')

# Change str -> float for numerical columns
df_ABR['Treatment $'] = df_ABR['Treatment $'].convert_objects(convert_numeric=True)
df_ABR['$/Hd/Rx'] = df_ABR['$/Hd/Rx'].convert_objects(convert_numeric=True)

# Remove percent signs from columns and convery str -> float
df_ABR['Treatment Success(%)'] = df_ABR['Treatment Success(%)'].apply(lambda x: float(x[:-1]))
df_ABR['Case Fatality Rate(%)'] = df_ABR['Case Fatality Rate(%)'].apply(lambda x: float(x[:-1]))